# Processing and filtering Variant Call Files to use with ADMIXTURE  

***

## Introduction  


A [Variant Calling Format file](https://en.wikipedia.org/wiki/Variant_Call_Format) stores sequence variations and flags (annotations) for each of those variations, including quality, coordinates respect to a reference, pair read information, etc. VCFs have two parts: a header and the variants. The header describes the format and meaning of flags and keeps track of the modifications that have been done to the VCF (like a log). It also keeps information about the reference genome used for calling the variants, dates, and versions of the software used to generate the VCF. The variant section contains all the variant nucleotides (non-variant nucleotides too if it is a genomic VCF), their position respect to the reference, quality flags, and allele information across all the samples included in the file. Every row in this section represents a single variant, and every column after the INFO and FORMAT columns represent a sample (with its allele for that variant).  


We first need to extract the samples of interest from a larger VCF with other samples, this means, we are filtering all variant rows and the sample columns that we want. We will use `vcftools` to do so. We will process the files and run all the analyses in a linux-like environment managed with [Anaconda (or Conda)](https://www.anaconda.com/products/individual). To run conda in a Windows machine, check out [the instructions](https://itsfoss.com/install-bash-on-windows/) to allow linux to run within Windows and to install Ubuntu on it. To know how to navigate to your regular folders from Ubuntu running inside Windows check [this](https://www.howtogeek.com/261383/how-to-access-your-ubuntu-bash-files-in-windows-and-your-windows-system-drive-in-bash/). Then you can install conda within Ubuntu using the Linux installer.  

We will create an environment to keep our software contained inside of it. Keeping environments for different tasks allows us to use different versions of software and libraries without conflict. It also allows us to save the environment and share it with others (and makes research reproducible).  

### 1. Create a conda environment with Python 3.7
We will create a conda environment with Python3.7. [Python](https://en.wikipedia.org/wiki/Python_(programming_language)) is a programming language with a lot of flexibility and applications. It is one fo the commonly used languages in bioinformatics (some use R too) and a lot of tools we will use are coded (or heavily use) python. We are using Python3 as oppose to Python2 which is now not supported. Once you have installed Conda, you can do:

```bash
conda create -n popgen python=3.7
```

Conda will show you a list of programs to install/update/downgrade. Conda does this to ensure that the versions of all the packages and libraries inside the environment generate as little conflict as possible. Just typo 'y' for conda to continue with the installation.  

To activate the environment you execute:

```bash
conda activate popgen
```

And to deactivate it:

```bash
conda deactivate
```

### 2. Install packages
To install packages it is easier to activate the conda environment and install it from 'inside'. More often than not, you don't know before hand which packages or programs you will be using (unless you are doing this for the tenth time). It is possible to create a conda environment from a list of packages, or even install someone else's conda environment, but that is not the focus of this exercise. So (we can also install a couple of other packages we will need):

```bash
conda activate popgen
conda install -c bioconda admixture
conda install -c bioconda plink
conda install -c bioconda bcftools
```

### 3. Filter VCF

Before filtering the VCF, it would be useful to have a look at it and get used to what it is and how it looks like. By using `less` you can 'enter' the file and navigate it. Use the arrows ot go up and down (or pgup and pg down). Exit by hiting 'q'. By using `grep` you can fetch the line to see the column headers in the variant section. If the VCF was correctly generated and it has multiple samples, you can see the names of the samples there.

```bash
less all_samples.vcf 
grep 'CHROM' all_samples.vcf
```

`bcftools` is a program to manipulate VCFs and generate summary statistics from the quality, mapping, and (to some extent) allele frequencies. We have a VCF with > 250 samples from which we need only a handful of samples. To get a VCF only with the samples we can specify a file with the samples we want to keep, one sample per row. To filter the VCF we do (don't forget to activate the environment):
Note: Due to mixedup names in a sample that we know occurred, we are not selecting the sample '52' but instead the '38a'

```bash
conda activate popgen
nano samples_to_keep.txt # copy paste the list, then crtl+x and y to save the file. Make sure the ending lines are Unix-like
bcftools view  --force-samples -S samples_to_keep.txt all_samples.vcf > filtered.vcf
grep 'CHROM' filtered.vcf # verify that the only samples in the file are the ones we want.
```

### 4. Changing formats

We will do a structure/admixture analysis using [ADMIXTURE](https://genome.cshlp.org/content/19/9/1655). ADMIXTURE uses as input a file in PLINK format. Converting a VCF into plink format generates two files: \*.ped and \*.map. \*.ped is a biallelic genotype table, \*.map is a map file with the location of every variant in \*.ped. To convert the VCF to plink format we use `plink`. The `--allow-extra-chr` argument tells plink to accept non-standard chromosome IDs (the case of non-model organisms mapped to draft assemblies). `--recode12` tells plink to transform the code for alleles from AGTC to 1 or 2, such that major alleles (the most common alleles) become 2 and minor alleles become 1. The rest of the arguments tell plink not to expect certain information that might not be available for our samples (sex, parental information, no family ID and no phenotype ID).

Because we filtered out some samples from the original VCF, it is possible that some genotype information is missing for all SNPs (there was information for those SNPs in the samples we removed but not in the samples we kept). We need to remove those SNPs too. The `--geno 0.99` argument removes the SNPs for which 99% of genotypes are missing.

```bash
plink --vcf filtered.vcf --allow-extra-chr --recode12 --geno 0.99 -out filtered_plink --allow-no-sex --no-sex --no-parents --no-fid --no-pheno
ls # have a look a the files generated
```


# Running ADMIXTURE
***

Running ADMIXTURE is relatively easy. `-j2` tells ADMIXTURE to run in two threads (a machine with an Intel Core i7 processor has 12 threads, always keep at least two unless you want to unfold a nightmare). It is always to set a known seed so that the analyses are fully reproducible. That is achieved with `--seed`. The bootstrap replicates are set inside the [] in the `-B` argument. After the arguments, we specify the input file and the number of populations to infer (one should run the analysis with a range of k values)

```bash
admixture --help # to have a look at the arguments and options.
admixture --cv -j2 --seed=666 filtered_plink.ped 4
```
But to run several iterations and several populations, we can use a loop in bash. Note that this script does not parallelise the analyses, simply launches the processes one by one, as they finish. There are ways of parallelising things in a regular machine, but it is advisable only if no other processes are running.

```bash
nano admixture.sh # creates a file. Copy paste the code below:

#!/bin/env bash
mkdir admixture_{1..6} # creates the files to store the results
echo "Program started at: `date`" >> admixture_stdout.out # prints to a file the date the loop starts running
for i in {1..6};
	do cd admixture_${i} # goes to approprite folder
    echo "RUNNING ADMIXTURE WITH `${i}` populations" >> ../admixture_stdout.out  # prints with cycle of the loop it is running
	admixture --cv -j2 --seed=666 filtered_plink.ped ${i} $2 >> ../admixture_stdout.out # prints admixture's screen output to the file
	wait # waits for the process to finish
	cd ../
done
echo "Program finished with exit code $? at: `date`" >> admixture_stdout.out # prints exit status

# close the file and save the changes with ctrl+x then 'y'
# change permissions (just in case) and execute:
chmod 777 ./admixture.sh
sh admixture.sh
```